## Arbeitsverzeichnis

In [1]:
# Aktuelles Arbeitsverzeichnis ermitteln
import os
os.getcwd()
# Arbeitspfad Privatrechner:
os.chdir("c:/Users/mhu/Documents/gitHub/magpie_chatbot")
# os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/magpie_langchain")

## Lade GPT, DB

In [2]:
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [3]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit 

db = SQLDatabase.from_uri("duckdb:///data/view_magpie.db") 

db.run("SHOW TABLES")

query = "SELECT * FROM view_daten_reichweite_menge;"
df = pd.read_sql(query, db._engine)
df

c:\Users\mhu\AppData\Local\miniconda3\envs\chatbot_magpie\lib\site-packages\duckdb_engine\__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


,daten_id,variable_id,variable_beschr,zeit_start,zeit_ende,zeit_einheit,wert,wert_einheit,reichweite_az,reichweite_id_list,reichweite_beschr_list,reichweite_id_beschr_list,reichweite_typ_list,typ_list,reichweite_klasse_list,klasse_list,reichweite_menge_id_list,menge_zugeh_reichweiten,quelle_list,tag_list
0,1,1,Interne FuE-Aufwendungen,1983-01-01,1983-12-31 23:59:59,Jahr,21809155,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Stifterverband | SV Wissenschaftsstatistik |...,| Forschung und Entwicklung | --- | Wirtschaft |
1,2,1,Interne FuE-Aufwendungen,1985-01-01,1985-12-31 23:59:59,Jahr,25629000,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Stifterverband | SV Wissenschaftsstatistik |...,| Forschung und Entwicklung | --- | Wirtschaft |
2,3,1,Interne FuE-Aufwendungen,1987-01-01,1987-12-31 23:59:59,Jahr,29212150,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Destatis | --- | Stifterverband | SV Wissens...,| Forschung und Entwicklung | --- | Wirtschaft |
3,4,1,Interne FuE-Aufwendungen,1989-01-01,1989-12-31 23:59:59,Jahr,32577985,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Stifterverband | SV Wissenschaftsstatistik |...,| Forschung und Entwicklung | FuE-Personal | -...
4,5,1,Interne FuE-Aufwendungen,1993-01-01,1993-12-31 23:59:59,Jahr,38624000,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Destatis | --- | Stifterverband | SV Wissens...,| Forschung und Entwicklung | FuE-Personal | -...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264779,288651,231,Studierende,2022-10-01,2023-03-31 23:59:59,Wintersemester,32,Anzahl,1.482197e-323,"5,735,1001",weiblich | Nicht deutsch | Zoll- Steuerverwaltung,[5] weiblich | [735] Nicht deutsch | [1001] Zo...,weiblich::Geschlecht | Nicht deutsch::Staatsan...,Geschlecht | Staatsangehörigkeit | Studienfach,weiblich::soziodemographische Merkmale | Nicht...,Hochschulbildung | soziodemographische Merkmal...,5::3,"weiblich, männlich",| Destatis |,None
264780,288652,231,Studierende,2022-10-01,2023-03-31 23:59:59,Wintersemester,47,Anzahl,9.881313e-324,"735,1001",Nicht deutsch | Zoll- Steuerverwaltung,[735] Nicht deutsch | [1001] Zoll- Steuerverwa...,Nicht deutsch::Staatsangehörigkeit | Zoll- Ste...,Staatsangehörigkeit | Studienfach,Nicht deutsch::soziodemographische Merkmale | ...,Hochschulbildung | soziodemographische Merkmale,None,None,| Destatis |,None
264781,288653,231,Studierende,2022-10-01,2023-03-31 23:59:59,Wintersemester,3363,Anzahl,9.881313e-324,"6,1001",männlich | Zoll- Steuerverwaltung,[6] männlich | [1001] Zoll- Steuerverwaltung,männlich::Geschlecht | Zoll- Steuerverwaltung:...,Geschlecht | Studienfach,männlich::soziodemographische Merkmale | Zoll-...,Hochschulbildung | soziodemographische Merkmale,6::3,"weiblich, männlich",| Destatis |,None
264782,288654,231,Studierende,2022-10-01,2023-03-31 23:59:59,Wintersemester,4022,Anzahl,9.881313e-324,"5,1001",weiblich | Zoll- Steuerverwaltung,[5] weiblich | [1001] Zoll- Steuerverwaltung,weiblich::Geschlecht | Zoll- Steuerverwaltung:...,Geschlecht | Studienfach,weiblich::soziodemographische Merkmale | Zoll-...,Hochschulbildung | soziodemographische Merkmale,5::3,"weiblich, männlich",| Destatis |,None


In [4]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print("-" * 40)

Tool Name: sql_db_query
Description: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
----------------------------------------
Tool Name: sql_db_schema
Description: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
----------------------------------------
Tool Name: sql_db_list_tables
Description: Input is an empty string, output is a comma-separated list of tables in the database.
----------------------------------------
Tool Name: sql_db_query_checker
Description: Use this tool to double check if your query is

## Generiere Tools

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents.agent_toolkits import create_retriever_tool
import ast
import re

##################################################################
# Generiere `rt_beschr_variable`
##################################################################

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


beschr_variable = query_as_list(db, "SELECT variable_beschr FROM view_daten_reichweite_menge")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(beschr_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_beschr_variable = create_retriever_tool(
    retriever,
    name="rt_beschr_variable",
    description=description,
)

##################################################################
# Generiere `rt_reichweite_variable`
##################################################################

reichweite_variable = query_as_list(db, "SELECT reichweite_beschr_list FROM view_daten_reichweite_menge")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(reichweite_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 15})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_reichweite_variable = create_retriever_tool(
    retriever,
    name="rt_reichweite_variable",
    description=description,
)

##################################################################
# Generiere `rt_beschr_wert_einheit`
##################################################################

werteinheit_variable = query_as_list(db, "SELECT wert_einheit FROM view_daten_reichweite_menge")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(werteinheit_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_werteinheit_variable = create_retriever_tool(
    retriever,
    name="rt_beschr_wert_einheit",
    description=description,
)

In [6]:
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

@tool
def variable_beschr(user_question: str) -> str:
    """
    Sucht nach variable_beschr in Tabelle
    """
    # GANZE Frage an den Retriever geben, nicht nur einen Ausschnitt
    matches = rt_beschr_variable.invoke(user_question)
    print(matches)
    if not matches:
        return "Error: Keine passende Variable gefunden."

    best_match = matches.split("\n")[0]  # Relevantes Ergebnis
    query = f"SELECT variable_beschr FROM view_daten_reichweite_menge WHERE variable_beschr = '{best_match}' LIMIT 1;"
    print(f"SQL Query (Variable): {query}")  # <<< HIER!
    result = db.run_no_throw(query)

    return result if result else "Error: Keine passende Variable gefunden."


@tool
def get_reichweite_beschr_list(user_question: str) -> str:
    """
    Sucht nach reichweite_beschr_list in Tabelle
    """
    # Verwende das bestehende Retriever-Tool, um die beste Übereinstimmung zu finden
    matches = rt_reichweite_variable.invoke(user_question)
    print("ergebnisse retriever:", matches)
    if not matches:
        return "Error: Keine passende Variable gefunden."

    prompt_template = """
    Gegeben ist eine Liste von Reichweiten:
    {matches}

    Welche Reichweite passt am besten zu dieser Frage: "{user_question}"?
    Gib mir nichts als die entsprechende Reichweite zurück.
    """

    prompt = PromptTemplate(input_variables=["matches", "user_question"], template=prompt_template)
    
    # Verwende RunnableSequence anstelle von LLMChain
    chain = prompt | llm
    result = chain.invoke(input={"matches": "\n".join(matches), "user_question": user_question})
    print("Ergebniss CHatgptabfrage:", result.content)

    # SQL-Abfrage mit der gefundenen besten Übereinstimmung
    query = f"SELECT reichweite_beschr_list FROM view_daten_reichweite_menge WHERE reichweite_beschr_list = '{result.content}' LIMIT 1;"
    print(f"SQL Query (Reichweite): {query}")  # <<< HIER!
    result = db.run_no_throw(query)

    return result if result else "Error: Keine passende Variable gefunden."

# @tool
# def get_wert_einheit_id(user_question: str) -> str:
#     """Sucht in der Tabelle 'wert_einheit' nach einer passenden ID basierend auf der Beschreibung."""
#     query = f"SELECT id FROM wert_einheit WHERE beschr LIKE '%{description}%' LIMIT 1;"
#     result = db.run_no_throw(query)
#     return result if result else "Error: Keine passende Werteinheit gefunden."

 


tools.extend([variable_beschr, get_reichweite_beschr_list])

In [7]:
test_output = variable_beschr.invoke("Wie viele Drittmittel von Stiftungen wurden 2020 in Deutschland ausgeschüttet")

print("Ergebnis:", test_output)

Drittmittel von Stiftungen

Drittmittel insgesamt

Drittmittel von Hochschulfördergesellschaften

Drittmittel der Länder (ohne Trägermittel)

Drittmittel von DFG-Einzelförderungen
SQL Query (Variable): SELECT variable_beschr FROM view_daten_reichweite_menge WHERE variable_beschr = 'Drittmittel von Stiftungen' LIMIT 1;
Ergebnis: [('Drittmittel von Stiftungen',)]


In [8]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1, "Die Anzahl der Nachrichten im Template ist nicht 1!"
# Bearbeite die bestehende Nachricht, indem du Text hinzufügst
prompt_template.messages[0].prompt.template += (
    "\nYou are Sparklehorse, a chatbot for the Stifterverband organization. "
    "Your primary task is to answer questions related to the Magpie database."
)

prompt_template.messages[0].pretty_print()

c:\Users\mhu\AppData\Local\miniconda3\envs\chatbot_magpie\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [9]:
system_message = prompt_template.format(
    dialect=db.dialect, 
    top_k=5
)

print(system_message)

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct duckdb query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see w

In [38]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Systemnachricht mit extra Anweisungen
suffix = (
    "Bevor eine SQL-Abfrage generiert wird, beachte folgendes:\n"
    "1. Benutze in Deinen Antworten immer der Werte von variable_beschr bzw get_reichweite_beschr_list, die du in deiner SQL abfrage verwendet hast, um irrtmümer zu vermeiden\n"
    "2. Verwende bei bedarf 'variable_beschr', um die korrekte variablen beschreibung basierend auf der Nutzeranfrage zu ermitteln und damit 'variable_beschr' zu filtern.\n"
    "3. Verwende bei bedarf 'get_reichweite_beschr_list', um die korrekte Reichweitenbeschreibung zu erhalten und damit 'get_reichweite_beschr_list' zu filtern.\n"
    "4. Um die Daten abzurufen, verwende immer die tabelle view_daten_reichweite_menge.\n"
    "5. Achte darauf, dass, wenn nach Daten zu Deutschland gefragt wird, Deutschland allein reichweite_beschr_list steht, also keine | oder anderen Werte\n"
    "6. Gib immer die finale SQL-Abfrage aus und erkläre sie. Rate niemals einen Wert oder eine ID — diese müssen immer mit den bereitgestellten Tools abgefragt werden.\n"
    "7. Falls eine ID nicht gefunden werden kann, weise darauf hin und erkläre, was fehlt.\n"
    "8. Filter Zeiträume auf diese Weise: date_part('year', zeit_start) = 2020"
)


system = f"{system_message}\n\n{suffix}"

# Neuen ReAct-Agent erstellen mit den vollständigen Tools
agent_executor = create_react_agent(llm, tools, state_modifier=system)


In [40]:
# Testanfrage an den Agenten
question = "Wieviele Vollzeitäqivalente gab es im Technische FuE-Personal 2021??"

for step in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}, 
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Wieviele Vollzeitäqivalente gab es im Technische FuE-Personal 2021??
================================== Ai Message ==================================
Tool Calls:
  variable_beschr (call_ffoH7tBuMHJAZHe8aTFnViu8)
 Call ID: call_ffoH7tBuMHJAZHe8aTFnViu8
  Args:
    user_question: Technische FuE-Personal
  get_reichweite_beschr_list (call_OWEScsxYWDVWH8xFPz0zSn4Q)
 Call ID: call_OWEScsxYWDVWH8xFPz0zSn4Q
  Args:
    user_question: Deutschland
Technisches FuE-Personal

Wissenschaftliches FuE-Personal

Sonstiges FuE-Personal

FuE-Personal

FuE-Personal ohne Wissenschaftler
SQL Query (Variable): SELECT variable_beschr FROM view_daten_reichweite_menge WHERE variable_beschr = 'Technisches FuE-Personal' LIMIT 1;
ergebnisse retriever: Deutschland

Deutschland | Ausland

Deutschland | Land (Träger)

Deutschland | Bund (Träger)

Deutschland | Oberstufenkurs

Deutschland | Privater Träger

Deutschland | Grundkurs (Ober

In [44]:
# Führe die Abfrage aus
query = """
SELECT wert, wert_einheit
FROM view_daten_reichweite_menge
WHERE variable_beschr = 'Technisches FuE-Personal'
  AND date_part('year', zeit_start) = 2021
LIMIT 5;
"""
result = db.run(query)



# Zeige das Ergebnis an
print(result)

[('169875', 'Mitarbeiter'), ('38070', 'Mitarbeiter'), ('146198', 'VZÄ'), ('32312', 'VZÄ'), ('682', 'VZÄ')]
